# **Lista de ejercicios 3 (30 de octubre, 2024)**
## Introducción a Machine Learning para CCSS

### Integrantes: 

- Victoria Olivera García (20171137)
- Víctor Manuel Raico Arce (F1092609)
- Fernando Mendoza Canal (20105246)
- Paolo Gutierrez Chochoca (F1120328)



#### **Instrucciones**

**1. Temas abordados:** Esta lista de ejercicios se enfoca en los temas: Tree Based Models &
Neural Networks.

**2. Formación de grupos:** Se permite la formación de grupos de hasta 5 integrantes.

**3. Puntuación de ejercicios:** La lista contiene 5 ejercicios. Cada ejercicio vale 4 puntos.

**4. Formato de entrega:** La resolución de los ejercicios debe presentarse en un archivo jupyter-notebook con todas las celdas ejecutadas.

**5. Fecha límite de entrega:** La fecha límite para la entrega es el miércoles 06 de noviembre a las 11:59 pm. Un representante del equipo debe subir su solucionario a la actividad correspondiente en la plataforma Canvas. Los nombres y códigos de todos los participantes deben ser incluidos en el solucionario.

### **Libraries**

In [121]:
#!pip install mlxtend

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

### **Pregunta 1**

#### Brinde una explicación detallada del algoritmo que se utiliza para implementar un **Árbol de Regresión**.

### **Pregunta 2**

#### Considere la dataset Carseats para predecir Sales como variable cuantitativa (puede encontrar el conjunto de datos [aquí](https://raw.githubusercontent.com/qlabpucp/datasets/main/datasets/carseats.csv)). Para ello:

#### **a)** Divida los datos en conjuntos de entrenamiento y de prueba

#### **b)** Ajuste un árbol de regresión al conjunto de entrenamiento. Interprete los resultados. <br>¿Qué valor de $MSE$ se obtiene para el conjunto de prueba?

#### **c)** Utilice el método **Bagging** para analizar estos datos. <br>¿Qué valor de $MSE$ obtiene para el conjunto de prueba?

#### **d)** Utilice el método **Random Forest** para analizar los datos. Repita el procedimiento descrito en el punto 3. <br>Describa el efecto de $m$, el número de variables consideradas en cada división, sobre la tasa de error obtenida. <br>Utilice la función $features\_importance()$ para determinar qué variables son las más importantes

#### **e)** Utilice el método **Gradient Boosting**. Repita el procedimiento descrito en el punto 4.

### **Pregunta 3**


#### Aplique los métodos **Boosting, Bagging y Random Forest** a una dataset de su elección. Asegúrese de dividir los datos en los conjuntos de entrenamiento y de prueba, ajustar los modelos sobre el conjunto de entrenamiento y evaluar su desempeño sobre el conjunto de prueba.

In [3]:
# Se usará el dataset College
ruta = "https://raw.githubusercontent.com/qlabpucp/datasets/main/datasets/college.csv"
data = pd.read_csv(ruta)
data = data.iloc[:, 1:]

# Codificando a valores binarios la variable 'Private'
mapa = {
    'Yes': 1,
    'No': 0
}

data['Private'] = data['Private'].map(mapa)

In [13]:
# Definiendo variables explicativas y variable explicada
X = data.drop(columns='Apps')
y = data['Apps']

# Dividiendo los datos entre train y test (30% para el data test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20241104)

# Escalando los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


#### ¿Cuán precisos son los resultados si los compara con métodos regresión lineal/logística y/o métodos de **Regularización**?

In [ ]:
# Para modelos Lasso y Ridge se ajusta primero los alpha óptimos:
# Definir el rango de alphas para Lasso y Ridge
alpha_values = {'alpha': [0.01, 0.1, 1, 10, 100]}

# Modelo Lasso con GridSearchCV
lasso_cv = GridSearchCV(Lasso(), param_grid=alpha_values, cv=5)
lasso_cv.fit(X_train, y_train)
best_lasso = lasso_cv.best_estimator_

# Modelo Ridge con GridSearchCV
ridge_cv = GridSearchCV(Ridge(), param_grid=alpha_values, cv=5)
ridge_cv.fit(X_train, y_train)
best_ridge = ridge_cv.best_estimator_

# Modelos
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': best_lasso,
    'Ridge': best_ridge,
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=20241104),
    'Bagging': BaggingRegressor(n_estimators=100, random_state=20241104),
    'Boosting': GradientBoostingRegressor(n_estimators=100, random_state=20241104)
}

In [19]:
# Entrenar y evaluar cada modelo
results = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    results[model_name] = {'MSE': mse, 'RMSE': rmse, 'R2': r2}

# Mostrar resultados
results_df = pd.DataFrame(results).T
results_df

,MSE,RMSE,R2
Linear Regression,1.479619e+06,1216.395955,0.863759
Lasso,1.415299e+06,1189.663551,0.869682
Ridge,1.477369e+06,1215.470580,0.863966
Random Forest,8.833714e+05,939.878407,0.918661
Bagging,8.615275e+05,928.185066,0.920672
Boosting,8.827712e+05,939.559055,0.918716


#### ¿Cuál de todos los modelos implementados muestra un mejor desempeño?

**Comentario:**

Según lo observado en el cuadro anterior, se tiene los siguientes resultados:
- El modelo *Bagging* muestra el menor RMSE (928.19), junto con el mayor $R^2$, lo que sugiere que es el modelo con mejor desempeño en esta comparación de modelos.
- Los modelos de *Random Forest* y *Boosting* están cerca del modelo *Bagging*.
- Los modelos de *regresión lineal* y de regularización (*Ridge* y *Lasso*) tienen un peor rendimiento en el RMSE y $R^2$.
- En conclusión, el modelo *Bagging* muestra el mejor desempeño dado que tiene el menor RMSE y mayor $R^2$ en comparación con los resultados de los otros modelos.

### **Pregunta 4**


#### Implemente una red neuronal con la database **Default** (puede encontrar el set de datos [aquí](https://raw.githubusercontent.com/qlabpucp/datasets/main/datasets/default.csv)). Use una capa oculta con 10 unidades y regularización por droput. Compare el desempeño de clasificación con respecto al de la regresión lineal.

### **Pregunta 5**


#### Considere la implementación del modelo de **Redes Neuronales Convolucionales** $(CNN)$ utilizando la dataset **CIFAR 100** presentada en el notebook de clases. En esta implementación inicial, se alcanzó un **Accuracy** aproximado de **0.44**. Su tarea es desarrollar una nueva implementación que supere este valor de **Accuracy**. Para lograrlo, se le anima a experimentar con los siguientes aspectos del modelo:

- **Arquitectura del Modelo:** Puede considerar la adición de más capas convolucionales y de pooling, incrementar el número de filtros por capa, o modificar el tamaño del kernel.

- **Regularización:** Experimente con diferentes tasas de dropout para prevenir el sobreajuste.

- **Entrenamiento:** Pruebe con distintos tamaños de batch que puedan influir en la convergencia del modelo durante el entrenamiento.

- **Optimización:** Evalúe el uso de diferentes algoritmos de optimización para verificar si mejoran la precisión del modelo.

#### La meta es ajustar estos parámetros para superar el **Accuracy** de **0.44** previamente alcanzado. Documente todas las modificaciones realizadas y los resultados obtenidos.